<h1>BEFORE STARTING</h1>

In this Capstone project, we will walk through solving a machine learning problem using a real-world dataset. The data has been obtained from the __*["Bureau of Transportation Statisitcs"](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time.)*__ which allows to download the flight data monthly at a time. In this capstone project we will focus on **the first half of 2017 domestic flights in the USA**. In this context, we downloaded 6 files corresponding to each month. Those files have been concatanated as **df_f (fligths DataFrame)**. The original datafiles has not been uploaded due to size of them.  The original dataset has __110 columns__, but some of them are irrelevant for this analysis so they will removed (such as Diversion times at each time etc) from the dataset. Additionally, international airline ID codes has been downloaded from __*[openflights](https://openflights.org/data.html)*__. Later on, this file was also merged with flight data file.

Note: Please keep in your mind that when you download the flight data file, click `"Prezipped File"` section on the website. The `explanations` of data (column names) also can be found on [the same website page]( https://www.transtats.bts.gov/Fields.asp?table_id=236)

In this study we focus on developing a model that can predict domestic flight cancellations, and then interpret the results to find the variables that are most predictive of the score.

This is a **supervised & classification machine learning task**: given a set of data with targets included, we want to train a model that can learn to map the features (also known as the explanatory variables) to the target.

```Supervised problem    :``` We are given both the features and the target

```Classification problem:``` The target is a __binary variable__, it takes either `0` or `1`
During training, we want the model to learn the relationship between the features and the target `Cancelled`. Then, to test how well the model has learned, we evaluate it on a testing set where it has never seen the answers!

<h1>IMPORTS</h1>

We will use the standard data science and machine learning library: `numpy` and `pandas`.

In [14]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Display up to 60 columns of a dataframe
pd.set_option('display.max_columns', None)

<h1>READING FILES & FIRST GLANCING THE DATASET</h1>

We will be loading our data into a pandas dataframe, one of the most useful data structures for data science. 

In [15]:
## df_f : flights dataframe
df_f = pd.DataFrame()

## each file reads relevant month's flight dataset
## to lessen the memory usage we will read files with some data types
data_types = {'Year':'int', 'Month':'int', 'DayofMonth':'int', 'AirlineID':'int','DepDelayMinutes':'float','WheelsOff':'float',\
              'FlightNum':'int','OriginAirportID':'int', 'OriginCityMarketID':'int','OriginStateFips':'int','WheelsOn':'float',\
              'OriginWac':'int','DestAirport':'int','DestStateFips':'int','DestWac':'int','DepDelay15':'float','TaxiOut':'float'}
    
## Reading files
file1 = pd.read_csv('On_Time_On_Time_Performance_2017_1.csv', low_memory = False, dtype=data_types) ## January-2017 Flight Data 
file2 = pd.read_csv('On_Time_On_Time_Performance_2017_2.csv', low_memory = False, dtype=data_types) ## February-2017 Flight Data 
file3 = pd.read_csv('On_Time_On_Time_Performance_2017_3.csv', low_memory = False, dtype=data_types) ## March-2017 Flight Data 
file4 = pd.read_csv('On_Time_On_Time_Performance_2017_4.csv', low_memory = False, dtype=data_types) ## April-2017 Flight Data 
file5 = pd.read_csv('On_Time_On_Time_Performance_2017_5.csv', low_memory = False, dtype=data_types) ## May-2017 Flight Data 
file6 = pd.read_csv('On_Time_On_Time_Performance_2017_6.csv', low_memory = False, dtype=data_types) ## June-2017 Flight Data 
airlines_ID = pd.read_csv('airline-id.csv', low_memory=False) ## International Airline_ID dataset

## Concatinating the flight datasets 
df_f = pd.concat([file1, file2, file3, file4, file5, file6])

In [16]:
## The sahpe of dataframe
df_f.shape

(2798209, 110)

In [17]:
df_f.Cancelled.value_counts()

0.0    2757668
1.0      40541
Name: Cancelled, dtype: int64

In [18]:
## Display top of the dataframe
df_f.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2017,1,1,17,2,2017-01-17,AA,19805,AA,N583AA,494,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1619,1616.0,-3.0,0.0,0.0,-1.0,1600-1659,17.0,1633.0,1837.0,5.0,1856,1842.0,-14.0,0.0,0.0,-1.0,1800-1859,0.0,NaN,0.0,277.0,266.0,244.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,1,1,18,3,2017-01-18,AA,19805,AA,N544AA,494,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1619,1614.0,-5.0,0.0,0.0,-1.0,1600-1659,13.0,1627.0,1815.0,6.0,1856,1821.0,-35.0,0.0,0.0,-2.0,1800-1859,0.0,NaN,0.0,277.0,247.0,228.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,1,1,19,4,2017-01-19,AA,19805,AA,N553AA,494,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1619,1611.0,-8.0,0.0,0.0,-1.0,1600-1659,17.0,1628.0,1824.0,2.0,1856,1826.0,-30.0,0.0,0.0,-2.0,1800-1859,0.0,NaN,0.0,277.0,255.0,236.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,1,1,20,5,2017-01-20,AA,19805,AA,N191AA,494,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1619,1656.0,37.0,37.0,1.0,2.0,1600-1659,18.0,1714.0,1926.0,3.0,1856,1929.0,33.0,33.0,1.0,2.0,1800-1859,0.0,NaN,0.0,277.0,273.0,252.0,1.0,1773.0,8,33.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,1,1,21,6,2017-01-21,AA,19805,AA,N170AA,494,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1619,1632.0,13.0,13.0,0.0,0.0,1600-1659,17.0,1649.0,1854.0,4.0,1856,1858.0,2.0,2.0,0.0,0.0,1800-1859,0.0,NaN,0.0,277.0,266.0,245.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In this dataframe, each row has information regarding a domestic `flight information`, which has origin airport, destintaion airport, Airline company name, departure flight time, arrival flight time, and other data regarding that flight. For the first half of 2017, there are approximateley `2,750,000` flights with `110` different parametrics in our dataset.

Taking a look at the raw data, we can already see a number of issues we need to solve. First of all, there are 110 columns and some of them have the same information. For example Origin State Name, State Name columns have the same information. Some columns are not related with cancellation such as columns linked with diversion of flights (which are not our scope for this study), so we will drop those at the very beginning and later we will continue to examine the columns whether they may offer benefical information for us analysis or not. If so not we will drop them later, too. 
Many columns have informations regarding diverted flight which is not our scope for this study. 


<h1>CLEANING THE DATASET</h1>

## **DROPPING COLUMNS REGARDING AIRPLANE OR AIRLINE COMPANY**

### UniqueCarrier (Unique Carrier Code)
It has the `same information with Carrier`, so we can drop it

In [19]:
df_f['UniqueCarrier'].equals(df_f['Carrier'])

True

### TailNum (Tail Number)
TailNumber is like car's plate number. It is `unique for each airplane`. We can see from the below table that an airplane can fly from different origins to different destionations. So it does not offer a spesific information for cancellation. We can drop it '

In [20]:
df_f[df_f.TailNum == 'N170AA'][['Dest', 'Origin', 'FlightDate']].head(10)

,Dest,Origin,FlightDate
4,PHX,CLT,2017-01-21
22,CLT,FLL,2017-01-21
32,CLT,RSW,2017-01-03
506,SFO,CLT,2017-01-01
905,PHX,DFW,2017-01-09
936,DFW,PHX,2017-01-09
1057,PHX,LAX,2017-01-06
1088,DFW,PHX,2017-01-06
1467,MCO,PHX,2017-01-09
3377,PHL,LAS,2017-01-05


### FlightNum (Flight Number)
It is `unique number for each flight` but it does not offer any information regarding cancellation, so we can drop it

###  Flights
Its `all values are 1 for all rows` and does not give us a spesific information, so we can drop it

In [21]:
df_f[['Flights', 'Origin', 'Dest', 'FlightDate']].head()

,Flights,Origin,Dest,FlightDate
0,1.0,CLT,PHX,2017-01-17
1,1.0,CLT,PHX,2017-01-18
2,1.0,CLT,PHX,2017-01-19
3,1.0,CLT,PHX,2017-01-20
4,1.0,CLT,PHX,2017-01-21


 ## **DROPPING COLUMNS REGARDING ORIGIN OF FLIGHT**

### OriginAirportID (Origin Airport, Airport ID)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

### OriginAirportSeqID (Origin Airport, Airport Sequence ID)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

### OriginCityMarketID (Origin Airport, Origin City, Market ID)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

### OriginStateFip  (Origin Airport, State Fip)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

### OriginStateName (Origin Airport, State Name)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

### OriginStateWac (Origin Airport, World Area Code)
It does not give us a spesific information and it is `linked with Origin`, so we can drop it

## **DROPPING COLUMNS REGARDING DESTINATION OF FLIGHT**

### DestinationAirportID (Destination Airport ID) 
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DestAirportSeqID (Destination Airport, Airport Sequence ID)
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DestCityMarketID (Destination Airport, Origin City, Market ID)
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DestStateFip (Destination State Fip)
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DestStateName (Destination State Name)
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DestWac (Destination Airport Wrold Area Code)
It does not give us a spesific information and it is `linked with Dest`, so we can drop it

### DistanceGroup (Distance Group)
It is just a distance classification and does not give us a spesific information, it is `linked with Origin and Dest`, so we can drop it

## **DROPPING COLUMNS REGARDING DIVERSION OF FLIGHT**

### Diverted (Diverted Flight Indicator (1=Yes))
It is related with `diverted flights` so we can drop them as well

### DivAirportLandings (Number of Diverted Airport Landings)
It is `linked with Diversion` and not in our interest, so we can drop it

### DivReachedDest (Diverted Flight Reaching Scheduled Destination Indicator (1=Yes))
It is `linked with Diversion` and not in our interest, so we can drop it

### DivActualElapsedTime (Elapsed Time of Diverted Flight Reaching Scheduled Destination, in Minutes)
It is `linked with Diversion` and not in our interest, so we can drop it

### DivArrDelay (Difference in Minutes Between Scheduled and Actual Arrival Time for a Diverted Flight Reaching Scheduled Destination)
It is `linked with Diversion` and not in our interest, so we can drop it

### DivDistance (Distance Between Scheduled Destination and Final Diverted Airport (miles))
It is `linked with Diversion` and not in our interest, so we can drop it

### Div1Airport, Div2Airport, Div3Airport, Div4Airport, Div5Airport (Diverted Airport Codes)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1AirportID, Div2AirportID, Div3AirportID, Div4AirportID, Div5AirportID (Diverted Airport ID)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1AirportSeqID, Div2AirportSeqID, Div3AirportSeqID, Div4AirportSeqID, Div5AirportSeqID (Airport Sequence ID of Diverted Airport)
They are `linked with Diversion` Diversion and not in our interest, so we can drop them

### Div1WheelsOn, Div2WheelsOn, Div3WheelsOn, Div4WheelsOn, Div5WheelsOn (Wheels On Time (local time: hhmm) at Diverted Airport)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1TotalGTime, Div2TotalGTime, Div3TotalGTime, Div4TotalGTime, Div5TotalGTime (Total Ground Time Away from Gate at Diverted Airport)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1LongestGTime, Div2LongestGTime, Div3LongestGTime, Div4LongestGTime, Div5LongestGTime (Longest Ground Time Away from Gate at Diverted Airport)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1WheelsOff, Div2WheelsOff, Div3WheelsOff, Div4WheelsOff, Div5WheelsOff (Wheels Off Time (local time: hhmm) at Diverted Airport)
They are `linked with Diversion` and not in our interest, so we can drop them

### Div1TailNum, Div2TailNum, Div3TailNum, Div4TailNum, Div5TailNum (Aircraft Tail Number for Diverted Airport)
They are `linked with Diversion` and not in our interest, so we can drop them

### Unnamed: 109
It is a `redundat column`, so we can drop it

In [22]:
redundant_columns = ['UniqueCarrier','TailNum','FlightNum','OriginAirportID','OriginAirportSeqID','OriginCityMarketID','OriginStateFips','OriginStateName','OriginWac','DestAirportID','DestAirportSeqID',                     
                   'DestCityMarketID','DestStateFips','DestStateName', 'DestWac','Flights','DistanceGroup','Diverted','DivAirportLandings','DivReachedDest','DivActualElapsedTime','DivArrDelay',
                   'DivDistance','Div1Airport','Div1AirportID','Div1AirportSeqID','Div1WheelsOn','Div1TotalGTime','Div1LongestGTime','Div1WheelsOff','Div1TailNum','Div2Airport','Div2AirportID',
                   'Div2AirportSeqID','Div2WheelsOn','Div2TotalGTime','Div2LongestGTime','Div2WheelsOff','Div2TailNum','Div3Airport','Div3AirportID','Div3AirportSeqID','Div3WheelsOn',
                   'Div3TotalGTime','Div3LongestGTime','Div3WheelsOff','Div3TailNum', 'Div4Airport','Div4AirportID','Div4AirportSeqID','Div4WheelsOn','Div4TotalGTime','Div4LongestGTime','Div4WheelsOff',
                   'Div4TailNum','Div5Airport','Div5AirportID','Div5AirportSeqID','Div5WheelsOn','Div5TotalGTime','Div5LongestGTime','Div5WheelsOff', 'Div5TailNum', 'Unnamed: 109']            

In [23]:
## Dropping unnecessary columns
df_f.drop(redundant_columns, axis=1, inplace=True)

In [24]:
df_f.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,AirlineID,Carrier,Origin,OriginCityName,OriginState,Dest,DestCityName,DestState,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
0,2017,1,1,17,2,2017-01-17,19805,AA,CLT,"Charlotte, NC",NC,PHX,"Phoenix, AZ",AZ,1619,1616.0,-3.0,0.0,0.0,-1.0,1600-1659,17.0,1633.0,1837.0,5.0,1856,1842.0,-14.0,0.0,0.0,-1.0,1800-1859,0.0,NaN,277.0,266.0,244.0,1773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,1,1,18,3,2017-01-18,19805,AA,CLT,"Charlotte, NC",NC,PHX,"Phoenix, AZ",AZ,1619,1614.0,-5.0,0.0,0.0,-1.0,1600-1659,13.0,1627.0,1815.0,6.0,1856,1821.0,-35.0,0.0,0.0,-2.0,1800-1859,0.0,NaN,277.0,247.0,228.0,1773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,1,1,19,4,2017-01-19,19805,AA,CLT,"Charlotte, NC",NC,PHX,"Phoenix, AZ",AZ,1619,1611.0,-8.0,0.0,0.0,-1.0,1600-1659,17.0,1628.0,1824.0,2.0,1856,1826.0,-30.0,0.0,0.0,-2.0,1800-1859,0.0,NaN,277.0,255.0,236.0,1773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,1,1,20,5,2017-01-20,19805,AA,CLT,"Charlotte, NC",NC,PHX,"Phoenix, AZ",AZ,1619,1656.0,37.0,37.0,1.0,2.0,1600-1659,18.0,1714.0,1926.0,3.0,1856,1929.0,33.0,33.0,1.0,2.0,1800-1859,0.0,NaN,277.0,273.0,252.0,1773.0,33.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,2017,1,1,21,6,2017-01-21,19805,AA,CLT,"Charlotte, NC",NC,PHX,"Phoenix, AZ",AZ,1619,1632.0,13.0,13.0,0.0,0.0,1600-1659,17.0,1649.0,1854.0,4.0,1856,1858.0,2.0,2.0,0.0,0.0,1800-1859,0.0,NaN,277.0,266.0,245.0,1773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
## After dropping some columns let's check new shape of the dataset
df_f.shape

(2798209, 46)

**We have dropped more than half of the columns and have more compact dataset with `46 columns` now.**</span>

<h4>CHECKING AIRPORTS</h4>

In [26]:
## Checking the number of Origin Airport
df_f.Origin.nunique()

315

In [27]:
## Checking the number of Destination Airport
df_f.Dest.nunique()

314

**There are 315 Origin and 314 Destination Airports in our dataset, we will check how much they have been used. **</span>

In [28]:
df_f.groupby("Origin")["Origin"].count().sort_values(ascending=False)


Origin
ATL    185547
ORD    128099
DEN    107949
LAX    105123
DFW     89759
SFO     82286
PHX     79372
LAS     75031
MCO     67221
IAH     65645
SEA     64702
MSP     64515
DTW     62210
BOS     61449
EWR     57149
CLT     55983
SLC     55585
BWI     48902
JFK     47533
FLL     46397
LGA     46034
MDW     42164
SAN     40144
DCA     37517
TPA     36274
PHL     36145
MIA     34662
DAL     34454
PDX     29401
STL     28402
        ...  
SJT       201
BGR       196
GCK       187
GUM       181
ALO       163
ACK       152
GRI       140
IFP       135
GUC       123
OTH       119
COU       117
TYR       114
MMH        93
TOL        76
TXK        75
WYS        59
JLN        57
PPG        57
ADK        52
UST        41
ABI        41
MVY        38
SUX        28
GST        27
AKN        27
DLG        27
HYA        16
GGG         8
BPT         7
TKI         1
Name: Origin, Length: 315, dtype: int64

In [29]:
df_f.groupby('Dest')['Dest'].count().sort_values(ascending=False)

Dest
ATL    185478
ORD    128047
DEN    107978
LAX    105124
DFW     89784
SFO     82305
PHX     79346
LAS     75031
MCO     67208
IAH     65608
SEA     64669
MSP     64502
DTW     62272
BOS     61453
EWR     57176
CLT     56006
SLC     55624
BWI     48910
JFK     47513
FLL     46380
LGA     46036
MDW     42164
SAN     40144
DCA     37527
TPA     36270
PHL     36154
MIA     34650
DAL     34453
PDX     29402
STL     28402
        ...  
STS       204
SJT       201
BGR       197
GCK       186
GUM       181
ALO       164
ACK       152
GRI       140
IFP       135
GUC       124
OTH       121
COU       118
TYR       114
MMH        93
TOL        76
TXK        75
WYS        59
JLN        58
PPG        57
ADK        52
UST        41
ABI        41
MVY        38
SUX        28
GST        27
AKN        27
DLG        27
HYA        16
GGG         8
BPT         7
Name: Dest, Length: 314, dtype: int64

When we check the airports majority of them have been used seldomly and compare the `top 20 aiports` in terms of flights numbers, we recognized that top 20 aiports are exactly the `same both in Origin and Destination`. In this context, only top 20 of the airports will be kept in our dataset.</span> 

In [30]:
## Top 20 airports have been kept in the major_airports dict

major_airports = dict(df_f.groupby('Dest')['Dest'].count().sort_values(ascending=False).head(20))

In [31]:
## DataFrame is filtered according to Origin and Dest Airports. Only top 20 airports have been kept in our analysis model

df_f = df_f[df_f['Dest'].isin(major_airports.keys()) & df_f['Origin'].isin(major_airports.keys())]

In [32]:
## New dataframe shape after filtering according to busiest airports

df_f.shape

(706981, 46)

New shape of dataset is reduced to about `700.000 rows` with `46 columns`.</span> 

<h2>MERGING FLIGHT DATA WITH AIRLINES_ID FILE</h2>

Airport File merged with flight Dataframe on `AirlineID` column

In [33]:
# We modified the Airline Company column name in its original file to merge it with the df_f dataFrame
airlines_ID.rename(columns={'Code':'AirlineID', 'Description':'Company'}, inplace=True)
df_f = pd.merge(df_f, airlines_ID)

In [34]:
#We can drop the AirLineID column now, becuse we will only need Company name in our dataset

df_f.drop(['AirlineID'], axis=1, inplace = True)

In [35]:
df_f.shape

(706981, 46)

In [36]:
df_f.Cancelled.value_counts()

0.0    697903
1.0      9078
Name: Cancelled, dtype: int64

In [37]:
9078/(706981)

0.012840514808743092

<h2>CONVERTING OUR MERGED DATASET INTO SINGLE FILE</h2>

In [80]:
df_f.to_csv('merged_flight_data.csv')

# Finally, we save the data frame into a csv file which can later be used for data exploration analysis and machine learning modeling.